## Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [2]:
%matplotlib inline

## Reshaping and pivoting

There are a number of fundamental operations for rearranging tabular data. These are alternatingly referred to as reshape or pivot operations.

### Reshaping with hierarchical indexing

Heirarchical indexing provides a consistent way to rearrange data into a DataFrame.
There are two primary actions:
    
- `stack`: this "rotates" or pivots form the columsn in the data to the rows
- `unstack`: this pivots from the rows into the columns

Less review these using a small dataframe

In [93]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [94]:
data.index

Index(['Ohio', 'Colorado'], dtype='object', name='state')

In [95]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [96]:
type(result)

pandas.core.series.Series

In [6]:
result.index

MultiIndex(levels=[['Ohio', 'Colorado'], ['one', 'two', 'three']],
           codes=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=['state', 'number'])

By default the innermost level is unstacked (same with `stack`). YOu can unstack a different level by passing a level number or name

In [97]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [8]:

result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [9]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values int he level aren't found in each of the subgroups.

In [10]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Stacking filters outmissing datay by default

In [98]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [12]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [99]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

When unstacking in a DataFrame, the level unstacked becomes the lowest level in the result.

In [14]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [15]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "long" to "wide" format

In [16]:
# Thecode here to get the data in a format that we want to work with
# Don't worry about each individual line

data = pd.read_csv('macrodata.csv')
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})


Data is frequently stored in databases in this *long format*. In the example below, `date` and `item` could be combined to create a primary key. 

In [17]:
ldata.head()

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


This makes it easier for database operations, but you will probably prefer to have a row for each timedate stamp and unique rows for each value type. DataFrame's `pivot` method performs exactly this transformation.

In [18]:
#the first two parameters are the columns to be used as the row 
# and column  index , and finally an optional value column to fill the DataFrame
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


Suppose you had two value columns that you wanted to reshape simultaneously:

In [19]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:5]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.204708
1,1959-03-31 23:59:59.999999999,infl,0.000,0.478943
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.519439
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.555730
4,1959-06-30 23:59:59.999999999,infl,2.340,1.965781


By ommitting the last argument, you obtain a DataFrame with hierarchical columns:

In [20]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.478943 -0.204708   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.965781 -0.555730   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.281746  0.092908   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.007189  1.246435   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.228913  0.274992   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.519439  
1959-06-30 23:59:59.999999999  1.393406  
1959-09-30 23:59:59.999999999  0.769023  
1959-12-31 23:59:59.999999999 -1.296221  
1960-03-31 23:59:59.999999999  1.352917

In [100]:
pivoted['value']['infl'][:5]

date
1959-03-31 23:59:59.999999999    0.00
1959-06-30 23:59:59.999999999    2.34
1959-09-30 23:59:59.999999999    2.74
1959-12-31 23:59:59.999999999    0.27
1960-03-31 23:59:59.999999999    2.31
Name: infl, dtype: float64

*Note*`pivot` is just a shortcut for creating a hierarchical index using `set_index` and reshaping with `unstack`.

In [22]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.478943 -0.204708   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.965781 -0.555730   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.281746  0.092908   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.007189  1.246435   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.228913  0.274992   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -2.001637  0.886429   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.438570  1.669025   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.519439  
1959-06-30 23:59:59.999999999  1.393406  
1959-09-30 23:59:59.999999999  0.769023  
1959-12-31 23:59:59.999999999 -1.296221  
1960-03-31 23:59:59.999999999  1.352917  
1960-06-30 23:59:59.999999999 -0.371843  
1960-09-30 23:59:59.999999999 -0.539741

## Data transformation

### Removing duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [102]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [103]:
# method that returns a boolean Series indicating whether each row 
# is a duplicate or not
data[data.duplicated()]

,k1,k2
1,one,1
4,two,3
6,two,4


In [104]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


If i call `data` again why are there still duplicates?

In [105]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [106]:
data.drop_duplicates(inplace=True)
data

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [107]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})

In [108]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [30]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### Transforming data using a function or mapping

In [31]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating tghe type of animal that each food came from. Below is a mapping with each distinct meat type to the kind of animal:

In [32]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

The `map` method on a Series accpets a function or a dict-like object containing a mapping.

Here we have a small problem that some of the meats above are capitalized and otehr are not. So we must convert each value to lower case:

In [33]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all of the work.

In [ ]:
def my_own(x):
    return x+'whatever I want'

In [34]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing values

Filling in missing data with `fillna` method can be thought of as a special case of a more general value replacement.

`replace` provides a simpler and more flexible way to do so.

In [110]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [112]:
# the first parameter is the value we are lookign for 
# the second is what we want to replace it with
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [113]:
# you can pass a list of values to be replaced
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [114]:
# you can also pass a list of values to be used as replacements
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [115]:
#you can also pass it a dict
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming axis indexes

In [116]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [117]:
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [118]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [119]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [120]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [121]:
# Always returns a reference to a DataFrame
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### Discretization and binning

In [122]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [123]:
cats = pd.DataFrame(ages, columns=['ages'])

In [124]:
cats

,ages
0,20
1,22
2,25
3,27
4,21
5,23
6,37
7,31
8,61
9,45


In [126]:
bins = [18, 25, 35, 60, 100]
cats['bins'] = pd.cut(cats.ages, bins)
cats

,ages,bins
0,20,"(18, 25]"
1,22,"(18, 25]"
2,25,"(18, 25]"
3,27,"(25, 35]"
4,21,"(18, 25]"
5,23,"(18, 25]"
6,37,"(35, 60]"
7,31,"(25, 35]"
8,61,"(60, 100]"
9,45,"(35, 60]"


In [127]:
pd.value_counts(cats.bins)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
Name: bins, dtype: int64

In [131]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']


In [132]:
cats['age_group'] = pd.cut(cats.ages, [18, 26, 36, 61, 100],labels=group_names)
cats.head()

,ages,bins,age_group
0,20,"(18, 25]",Youth
1,22,"(18, 25]",Youth
2,25,"(18, 25]",Youth
3,27,"(25, 35]",YoungAdult
4,21,"(18, 25]",Youth


If you don't have predetermined bins, you can give the an integer instead and `cut` will create equal length bins based on the min and max values  

In [53]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.25, 0.49], (0.25, 0.49], (0.73, 0.98], (0.25, 0.49], (0.25, 0.49], ..., (0.25, 0.49], (0.73, 0.98], (0.49, 0.73], (0.49, 0.73], (0.49, 0.73]]
Length: 20
Categories (4, interval[float64]): [(0.0032, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.98]]

Additionally you can use `qcut` to break your data into quantiles

In [54]:
data = np.random.randn(1000) # Normally distributed
rand_data = pd.qcut(data, 4) # Cut into quartiles
rand_data

[(0.636, 3.26], (-3.746, -0.648], (0.636, 3.26], (-0.022, 0.636], (-0.648, -0.022], ..., (0.636, 3.26], (-0.022, 0.636], (-3.746, -0.648], (-0.022, 0.636], (-0.022, 0.636]]
Length: 1000
Categories (4, interval[float64]): [(-3.746, -0.648] < (-0.648, -0.022] < (-0.022, 0.636] < (0.636, 3.26]]

In [55]:
pd.value_counts(rand_data)

(0.636, 3.26]       250
(-0.022, 0.636]     250
(-0.648, -0.022]    250
(-3.746, -0.648]    250
dtype: int64

In [56]:
#cut the data at your own predetermined quantiles
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.022, 1.298], (-3.746, -1.274], (-0.022, 1.298], (-0.022, 1.298], (-1.274, -0.022], ..., (-0.022, 1.298], (-0.022, 1.298], (-3.746, -1.274], (-0.022, 1.298], (-0.022, 1.298]]
Length: 1000
Categories (4, interval[float64]): [(-3.746, -1.274] < (-1.274, -0.022] < (-0.022, 1.298] < (1.298, 3.26]]

### Computing indicator / dummy variables

In [130]:
cats.head()

,ages,bins
0,20,"(18, 25]"
1,22,"(18, 25]"
2,25,"(18, 25]"
3,27,"(25, 35]"
4,21,"(18, 25]"


In [133]:
dummies = pd.get_dummies(cats['age_group'], prefix='age_')
cats_with_dummy = cats.join(dummies)
cats_with_dummy.describe()

,ages,age__Youth,age__YoungAdult,age__MiddleAged,age__Senior
count,12.000000,12.000000,12.000000,12.000000,12.0
mean,32.083333,0.416667,0.250000,0.333333,0.0
std,12.198795,0.514929,0.452267,0.492366,0.0
min,20.000000,0.000000,0.000000,0.000000,0.0
25%,22.750000,0.000000,0.000000,0.000000,0.0
50%,29.000000,0.000000,0.000000,0.000000,0.0
75%,38.000000,1.000000,0.250000,1.000000,0.0
max,61.000000,1.000000,1.000000,1.000000,0.0


### Detecting and filtering outliers

In [134]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [60]:
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [61]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


Values can easily be set based on these criteria. Here is code to cap values outside the interval of -3 to 3:

In [136]:
#np.sign returns -1 if the value is <0 and 1 if the value is >0
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


## String manipulation

### String object methods

In [63]:
val = 'a,b,  lassie'
val.split(',')

['a', 'b', '  lassie']

In [64]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'lassie']

In [65]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::lassie'

In [66]:
'::'.join(pieces)

'a::b::lassie'

In [71]:
'lassie' in val

True

In [68]:
val.index(',')

1

In [72]:
val.find(':')

-1

In [73]:
val.count(',')

2

In [74]:
val.replace(',', '::')

'a::b::  lassie'

In [75]:
val.replace(',', '')

'ab  lassie'

### Vectorized string functions in pandas

In [76]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [77]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [78]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [79]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

## Example: USDA Food Database

The US Department of Agriculture makes available a database of food nutrient information. Ashley Williams, an English hacker, has made available a version of this database in JSON format (http://ashleyw.co.uk/project/food-nutrient-database). The records look like this:

In [82]:
import json
db = json.load(open('foods-2011-10-03.json'))
len(db)

6636

In [83]:
db[0]

{'id': 1008,
 'description': 'Cheese, caraway',
 'tags': [],
 'manufacturer': '',
 'group': 'Dairy and Egg Products',
 'portions': [{'amount': 1, 'unit': 'oz', 'grams': 28.35}],
 'nutrients': [{'value': 25.18,
   'units': 'g',
   'description': 'Protein',
   'group': 'Composition'},
  {'value': 29.2,
   'units': 'g',
   'description': 'Total lipid (fat)',
   'group': 'Composition'},
  {'value': 3.06,
   'units': 'g',
   'description': 'Carbohydrate, by difference',
   'group': 'Composition'},
  {'value': 3.28, 'units': 'g', 'description': 'Ash', 'group': 'Other'},
  {'value': 376.0,
   'units': 'kcal',
   'description': 'Energy',
   'group': 'Energy'},
  {'value': 39.28,
   'units': 'g',
   'description': 'Water',
   'group': 'Composition'},
  {'value': 1573.0, 'units': 'kJ', 'description': 'Energy', 'group': 'Energy'},
  {'value': 0.0,
   'units': 'g',
   'description': 'Fiber, total dietary',
   'group': 'Composition'},
  {'value': 673.0,
   'units': 'mg',
   'description': 'Calcium,

In [84]:
db[0].keys()

dict_keys(['id', 'description', 'tags', 'manufacturer', 'group', 'portions', 'nutrients'])

In [85]:
DataFrame(db)

,description,group,id,manufacturer,nutrients,portions,tags
0,"Cheese, caraway",Dairy and Egg Products,1008,,"[{'value': 25.18, 'units': 'g', 'description':...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}]",[]
1,"Cheese, cheddar",Dairy and Egg Products,1009,,"[{'value': 24.9, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'cup, diced', 'grams': ...",[]
2,"Cheese, edam",Dairy and Egg Products,1018,,"[{'value': 4.22, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...",[]
3,"Cheese, feta",Dairy and Egg Products,1019,,"[{'value': 5.2, 'units': 'g', 'description': '...","[{'amount': 1, 'unit': 'cup, crumbled', 'grams...",[]
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,,"[{'value': 3.27, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}]",[]
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,,"[{'value': 3.72, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'cup, diced', 'grams': ...",[]
6,"Cheese, romano",Dairy and Egg Products,1038,,"[{'value': 31.8, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...",[]
7,"Cheese, roquefort",Dairy and Egg Products,1039,,"[{'value': 21.54, 'units': 'g', 'description':...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...",[]
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,,"[{'value': 16.41, 'units': 'g', 'description':...","[{'amount': 1, 'unit': 'cup, diced', 'grams': ...",[]
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,,"[{'value': 2.96, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'cup', 'grams': 242.0},...",[]


In [86]:
db[0]['nutrients'][0]

{'value': 25.18,
 'units': 'g',
 'description': 'Protein',
 'group': 'Composition'}

In [87]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [88]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [89]:
info[:5]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [90]:
info

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [91]:
pd.value_counts(info.group)[:10]

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Legumes and Legume Products          365
Fast Foods                           365
Lamb, Veal, and Game Products        345
Sweets                               341
Pork Products                        328
Fruits and Fruit Juices              328
Name: group, dtype: int64

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
result['Zinc, Zn'].sort_values().plot(kind='barh')